In [1]:
import os,sys,tqdm
import numpy as np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import *

from collections import Counter
import pandas as pd
import shutil
import pickle
import gc
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def basic_statistics(all_length):
    '''
    input: length list of elements e.g.[1,1,1,3,5,9,4,2,1,3,54,78,5...]
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

In [2]:
max_words = 8352#8352 #Top most frequent words to consider. Any less frequent word will appear as oov_char value in the sequence data.
max_length = 360#360

In [3]:
word_index = reuters.get_word_index()
print('all_words#:',len(word_index))
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,maxlen=max_length,
                                                         test_split=0.2,seed=830913)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
word_index

all_words#: 30979
8260 train sequences
2066 test sequences


{'mdbl': 10996,
 'fawc': 16260,
 'degussa': 12089,
 'woods': 8803,
 'hanging': 13796,
 'localized': 20672,
 'sation': 20673,
 'chanthaburi': 20675,
 'refunding': 10997,
 'hermann': 8804,
 'passsengers': 20676,
 'stipulate': 20677,
 'heublein': 8352,
 'screaming': 20713,
 'tcby': 16261,
 'four': 185,
 'grains': 1642,
 'broiler': 20680,
 'wooden': 12090,
 'wednesday': 1220,
 'highveld': 13797,
 'duffour': 7593,
 '0053': 20681,
 'elections': 3914,
 '270': 2563,
 '271': 3551,
 '272': 5113,
 '273': 3552,
 '274': 3400,
 'rudman': 7975,
 '276': 3401,
 '277': 3478,
 '278': 3632,
 '279': 4309,
 'dormancy': 9381,
 'errors': 7247,
 'deferred': 3086,
 'sptnd': 20683,
 'cooking': 8805,
 'stratabit': 20684,
 'designing': 16262,
 'metalurgicos': 20685,
 'databank': 13798,
 '300er': 20686,
 'shocks': 20687,
 'nawg': 7972,
 'tnta': 20688,
 'perforations': 20689,
 'affiliates': 2891,
 '27p': 20690,
 'ching': 16263,
 'china': 595,
 'wagyu': 16264,
 'affiliated': 3189,
 'chino': 16265,
 'chinh': 16266,
 '

In [13]:
# train_len = [len(x) for x in x_train]
# test_len = [len(x) for x in x_test]
# all_len = train_len
# all_len.extend(test_len)
# basic_statistics(all_len)

,length
mean,145.964197
std,145.878476
mode,17.000000
min,2.000000
q1,60.000000
median,95.000000
q3,180.000000
max,2376.000000
iqr,120.000000
outlier,360.000000


In [15]:
# df = pd.DataFrame(all_len)
# df.to_excel('./results/length_dist.xlsx', header=False, index=False)
# df

In [21]:
# train_words = []
# for x in x_train:
#     train_words.extend(x)
# test_words = []
# for x in x_test:
#     test_words.extend(x)
# all_words = train_words
# all_words.extend(test_words)
# all_statistcs = Counter(all_words)
# all_statistcs

Counter({1: 11228,
         53: 4213,
         352: 647,
         26: 8451,
         14: 15015,
         279: 801,
         39: 5818,
         72: 3091,
         4497: 26,
         18: 11039,
         83: 2597,
         5291: 21,
         88: 2381,
         5397: 20,
         11: 20141,
         3412: 37,
         19: 10755,
         151: 1363,
         230: 962,
         831: 253,
         15: 13329,
         165: 1232,
         318: 707,
         3780: 33,
         124: 1676,
         1527: 117,
         1424: 128,
         35: 6588,
         5302: 20,
         12: 16668,
         17: 11191,
         486: 459,
         341: 663,
         142: 1466,
         255: 870,
         219: 997,
         429: 528,
         68: 3363,
         146: 1402,
         252: 882,
         191: 1098,
         15448: 3,
         3631: 35,
         2283: 65,
         71: 3120,
         10: 29581,
         342: 660,
         49: 4565,
         1977: 80,
         324: 695,
         27: 8311,
         9222: 

In [28]:
# df = pd.DataFrame.from_dict(dict(all_statistcs), orient = 'index')
# df.to_excel('./results/words_dist2.xlsx', header=False, index=True)
# df

,0
1,11228
53,4213
352,647
26,8451
14,15015
...,...
24452,1
18567,2
27222,1
26864,1


In [4]:
trainX = tf.keras.preprocessing.sequence.pad_sequences(x_train,maxlen=max_length,padding='post',value=0)
testX = tf.keras.preprocessing.sequence.pad_sequences(x_test,maxlen=max_length,padding='post',value=0)
print(trainX.shape, testX.shape)

(8260, 360) (2066, 360)


In [5]:
hidden_dim = 128
# do = 0.1

## Graph execution
### Embedder

In [42]:
int_id = Input(shape=(max_length,), dtype='int32', name='int_ids') # 輸入的api funvtion name ID
int_ids = Masking(mask_value=0)(int_id)
sent_emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')(int_ids) 

### Encoder

In [43]:
rnn = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)(sent_emb)
rnn = BatchNormalization(name='bn')(rnn)


### Filter

In [44]:
fil = TimeDistributed(Dense(1,activation='sigmoid',
                             name='filter_out'),name='TD2')(rnn)

### Classfier

In [47]:
mul = Multiply()([fil,sent_emb])
clf = LSTM(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm')(mul)
clf = BatchNormalization(name='bn3')(clf)
clf = Dense(max(y_train)+1,activation='softmax',name='clf')(clf)

## Compile

In [48]:
model = Model(inputs=int_id, outputs = clf)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
int_ids (InputLayer)            [(None, 358)]        0                                            
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 358)          0           int_ids[0][0]                    
__________________________________________________________________________________________________
glove_emb (Embedding)           (None, 358, 128)     1280000     masking_1[0][0]                  
__________________________________________________________________________________________________
common_extract (GRU)            (None, 358, 64)      37248       glove_emb[0][0]                  
______________________________________________________________________________________________

In [53]:
# loss
import keras.backend as K
def custom_objective(layer):
    return K.sum(layer.output)
#     return K.sum(layer.output)
# kk = tf.keras.backend.ea
model.compile(loss=custom_objective(model.get_layer(name='TD2')),optimizer='adam')

Using TensorFlow backend.


OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.

## Eager Execution

In [52]:
# whole model
do = 0
init = tensorflow.keras.initializers.Ones()
class base_model(Model):
    def __init__(self):
        super(base_model, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')
        self.rnn1 = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.bn1 = BatchNormalization(name='bn1')
        self.fil = Dense(1,activation='hard_sigmoid',kernel_initializer=init,bias_initializer=init,name='filter_out')
        #self.fil = TimeDistributed(Dense(1,activation='sigmoid', name='filter_out'),name='TD2')
        self.mul = Multiply()
        self.rnn2 = Bidirectional(GRU(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = LSTM(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')
    def transform(self,x):
        return tf.math.round(x)
    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        x = self.rnn1(x1)
        x = self.bn1(x)
        y = self.fil(x)
        y1 = self.transform(y)
        x2 = self.mul([y1,x1])
        x = self.rnn2(x2) #x
        x = self.bn2(x)
        y2 = self.out(x)
        return y,y1,y2
        #return y,y1,y2,x2
        
model = base_model()

In [1009]:
# partial1 model
# init_w = tensorflow.keras.initializers.Constant(value=1.667) #portyion=0.6, w=0.9, b = 0.8-0.85 (0.83從0開始)
init_w = tensorflow.keras.initializers.RandomNormal(mean=1.667,stddev=0.33)
# init_b = tensorflow.keras.initializers.Constant(value=1.667) #w=1 ; b=0.499, portion=1
init_b = tensorflow.keras.initializers.RandomNormal(mean=1.667,stddev=0.33)
def onezero(x):
    portion = 0.6#0.6#0.6 #0.6~1
    z = tf.where(x>=1.0, x - x + 1.0, x)
    y = tf.where(z<=0.0, z - z + 0.0, portion*z)
    return y

class base_model_1(Model):
    def __init__(self):
        super(base_model_1, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')
        self.rnn1 = GRU(int(hidden_dim/4),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.att = Attention(name='selfatt')
        self.bn1 = BatchNormalization(name='bn1')
        #self.fil = Dense(1,activation=onezero,name='filter_out')
        self.fil = TimeDistributed(Dense(1,activation=onezero,kernel_initializer=init_w,bias_initializer=init_b, name='filter_out'),name='TD2') #relu/linear/step function

    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        x = self.att([x1,x1])
        #x = self.rnn1(x1)
        #x = self.bn1(x)
        y = self.fil(x)
        return x1,y

model1 = base_model_1()
#phase2
# model3 = load_model(saveP)

# model3 = base_model_1()
# model3.load_weights('./model/2019110501/model1')#,by_name=True)
# model1.emb.set_weights(model3.emb.get_weights())

In [1010]:
# partial2 model
class base_model_2(Model):
    def __init__(self):
        super(base_model_2, self).__init__()
        self.mul = Multiply()
        self.rnn2 = Bidirectional(GRU(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = GRU(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')

    def call(self,x1,y1):
        x2 = self.mul([y1,x1])
        x = self.rnn3(x2) #x2 #y1=weight|binary
        x = self.bn2(x)
        y2 = self.out(x)
        return y2
    
model2 = base_model_2()

In [448]:
'''
# TEST
x = tf.random.uniform((32, 6))
out1,out2 = model1(x)
out2 = tf.squeeze(out2,axis=-1)
# out2 = out2.astype('float32')
# out2 = tf.dtypes.cast(out2, tf.float8)
# print(out2.shape)
yy = tf.matmul(out2,kk)
print(yy[0].shape)
tf.where(yy[0]==0,1,0)

gg = tf.range(-5,5)
gg = tf.expand_dims(gg,axis=0)
gg = tf.keras.backend.repeat_elements(gg,rep=32,axis=0)
tf.where(gg==1,gg,0)

kk = tf.Variable(np.array([[1.0,1.0,1.0,0.0,0.0,0.0],[0.0,1.0,1.0,1.0,0.0,0.0],[0.0,0.0,1.0,1.0,1.0,0.0],
                           [0.0,0.0,0.0,1.0,1.0,1.0]]).T,dtype='float32')
kk = tf.expand_dims(kk,axis=0)
kk = tf.keras.backend.repeat_elements(kk,rep=1,axis=0)
kk.shape
'''

(32, 4)


<tf.Tensor: id=5770753, shape=(32, 4), dtype=int32, numpy=
array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]], dtype=int32)>

In [896]:
#記得要跑到一個好的data
batch_size = 128 #,reshuffle_each_iteration=True
train_ds = tf.data.Dataset.from_tensor_slices((trainX,y_train)).shuffle(trainX.shape[0]).batch(batch_size)
valid_ds = tf.data.Dataset.from_tensor_slices((testX,y_test)).batch(batch_size)

In [1011]:
seq_num = 2 #連續幾個才叫做連續，要改modify_idx看有幾個

arr_len = max_length - seq_num + 1
seq_arr = []
for i in range(arr_len):
    ori_np = np.array([0]*max_length)
    modify_idx = [i,i+1] #要跟著seq_num改
    ori_np[modify_idx]=1
    seq_arr.append(ori_np)
seq_arr = np.array(seq_arr)
seq_mask = tf.Variable(seq_arr.T,dtype='float32')
seq_mask = tf.expand_dims(seq_mask,axis=0)
seq_mask

<tf.Tensor: id=14460211, shape=(1, 360, 359), dtype=float32, numpy=
array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 1., 0.],
        [0., 0., 0., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 1.]]], dtype=float32)>

In [1012]:
def loss_object1(predictions): #filter loss
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def one_percentage(predictions): #1 num
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def seq_loss(predictions):
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    predictions = tf.squeeze(predictions,axis=-1)
    results = tf.matmul(predictions,seq_mask)
    results = tf.where(results==seq_num,1.0,0.0)
    loss_ = tf.reduce_mean(results)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy() #clf loss

optimizer1 = tf.keras.optimizers.Nadam(learning_rate=0.001)
optimizer2 = tf.keras.optimizers.RMSprop()

train_loss = tf.keras.metrics.Mean(name='train_loss') #total_loss
train_accloss = tf.keras.metrics.Mean(name='train_accloss')#loss_acc
train_filloss = tf.keras.metrics.Mean(name='train_filloss') #loss_filter
train_seqloss = tf.keras.metrics.Mean(name='train_seqloss') #loss_seq
train_ones = tf.keras.metrics.Mean(name='train_ones') #ones_num
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') #acc_rate

test_loss = tf.keras.metrics.Mean(name='test_loss') #total_loss
test_accloss = tf.keras.metrics.Mean(name='test_accloss')#loss_acc
test_filloss = tf.keras.metrics.Mean(name='test_filloss') #loss_filter
test_seqloss = tf.keras.metrics.Mean(name='test_seqloss')
test_ones = tf.keras.metrics.Mean(name='test_ones') #ones_num
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [1013]:
# seperate partial model
alpha = 0.00001 #pahse1: -0.1 / 0.0 ; phase2: 0.01~0.05~0.1 有1-就是希望1越多，沒1-就是希望0越多1越少
beta = 1.0 #clf loss 越大越要求分好
gamma = 0.0 #seqloss 越大越要求連續
#-0.001 / 1.0 / 1.0

@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=False) as tape:
        emb, pred_imp = model1(x)
        #loss1 = alpha*loss_object1(pred_imp) #phase1
        #pred_imp2 = tf.math.round(pred_imp)
        #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
        pred_imp2 = tf.math.round(pred_imp)
        loss1 = loss_object1(pred_imp) #有1-就是希望1越多，沒1-就是希望0越多1越少 #pahse2:alpha*loss_object1(pred_imp) ; phase1: alpha*(1-loss_object1(pred_imp))
        pred_cat = model2(emb,pred_imp2) #pahse1: pred_imp; phase2; pred_imp2
        loss2 = loss_object2(yc, pred_cat)
        loss3 = 1-seq_loss(pred_imp2)
        loss = alpha*loss1 + beta*loss2 + gamma*loss3
    trainable_variable = model1.trainable_variables
    trainable_variable.extend(model2.trainable_variables)
    gradients = tape.gradient(loss,trainable_variable)
    optimizer1.apply_gradients(zip(gradients,trainable_variable))
    
    train_loss(loss) #total_loss
    train_filloss(loss1)
    train_accloss(loss2)
    train_seqloss(loss3) #loss_seq
    train_accuracy(yc, pred_cat) #acc_rate
    ones = one_percentage(pred_imp2) #pred_imp2
    train_ones(ones) #ones_num
    
    
@tf.function
def test_step(x,yc):
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #phase1: pred_imp ; phase2:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)
    

In [331]:
'''
#AIO
alpha = 0.1
beta = 1
gamma = 0
@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=True) as tape: #persistent=True
        pred_imp,pred_round , pred_cat = model(x)
#         pred_cat = model(x)
#         loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat)
        loss1 = alpha*loss_object1(pred_imp)
        loss2 = beta*loss_object2(yc,pred_cat)
        loss = loss_object2(yc,pred_cat)
#     gradients = tape.gradient(loss, model.trainable_variables)
    grad1 = tape.gradient(loss1, model.trainable_variables)
    grad2 = tape.gradient(loss2, model.trainable_variables)
#     optimizer1.apply_gradients(zip(gradients, model.trainable_variables))
    optimizer1.apply_gradients(zip(grad1, model.trainable_variables))
    optimizer2.apply_gradients(zip(grad2, model.trainable_variables))
#     with tf.GradientTape() as tape:
#         pred_imp , pred_cat = model(x)
#         loss2 = loss_object2(yc,pred_cat)
#         loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat)
#     grad2 = tape.gradient(loss2, model.trainable_variables)
#     optimizer2.apply_gradients(zip(grad2, model.trainable_variables))

    train_loss(loss)
    train_accuracy(yc, pred_cat)
    ones = one_percentage(pred_round)
    train_ones(ones)
    
@tf.function
def test_step(x,yc):
    pred_imp,pred_round, pred_cat = model(x)
#     pred_cat = model(x)
    t_loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat) 
#     t_loss = loss_object2(yc,pred_cat)
    
    test_loss(t_loss)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_round)
    test_ones(t_ones)
'''    

In [1014]:
# EPOCHS = 2000
DateID = '2019111101'

saveP1 = './model/'+DateID+'/model1'
saveP2 = './model/'+DateID+'/model2'
train_loss_acc = []
train_loss_filter = []
train_loss_seq = []
train_weighted_loss = []
train_acc_rate = []
train_ones_num = []

test_loss_acc = []
test_loss_filter = []
test_loss_seq = []
test_weighted_loss = []
test_acc_rate = []
test_ones_num = []

# signature_dict = {'att':model1.att}

gc.collect()
best_clf = 0.0
for epoch in range(EPOCHS):
    for text, labels in train_ds:
        train_step(text, labels)

    for test_text, test_labels in valid_ds:
        test_step(test_text, test_labels)

    template = 'Epoch {}, Total Loss: {}, Clf Loss: {}, Filter Loss: {}, Seq Loss: {}, Accuracy Rate: {:5.2f}%, Ones Portion: {}, \
            Test_Total_Loss: {}, Test_Clf_Loss: {}, Test_Filter_Loss: {}, TEST_Seq_Loss: {}, Test_Accuracy_Rate: {:5.2f}%, Test_Ones_Portion: {}'
    print(template.format(epoch+1,train_loss.result(),
                          train_accloss.result(),train_filloss.result(),train_seqloss.result(),
                        train_accuracy.result()*100,train_ones.result(),
                        test_loss.result(),
                        test_accloss.result(),test_filloss.result(),test_seqloss.result(),
                        test_accuracy.result()*100,test_ones.result(),
                        ))

    train_loss_acc.append( train_accloss.result().numpy())
    train_loss_filter.append( train_filloss.result().numpy())
    train_loss_seq.append( train_seqloss.result().numpy())
    train_weighted_loss.append( train_loss.result().numpy())
    train_acc_rate.append( train_accuracy.result().numpy())
    train_ones_num.append( train_ones.result().numpy())
    
    test_loss_acc.append( test_accloss.result().numpy())
    test_loss_filter.append( test_filloss.result().numpy())
    test_loss_seq.append( test_seqloss.result().numpy())
    test_weighted_loss.append( test_loss.result().numpy())
    test_acc_rate.append( test_accuracy.result().numpy())
    test_ones_num.append( test_ones.result().numpy())
    if best_clf<=test_accuracy.result()*100:
        #tf.saved_model.save(model1,saveP1+'_all')
        #model1.save(saveP1,save_format='h5')
        #model2.save(saveP2,save_format='h5')
        #tf.saved_model.save(model2,saveP2+'_all')
        model1.save_weights(saveP1,save_format='tf')
        model2.save_weights(saveP2,save_format='tf')
        best_clf = test_accuracy.result()*100
        print('===MODEL WEIGHTS SAVED===',saveP1,saveP2)
    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accloss.reset_states()
    train_filloss.reset_states()
    train_seqloss.reset_states()
    train_ones.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accloss.reset_states()
    test_filloss.reset_states()
    test_seqloss.reset_states()
    test_ones.reset_states()
    test_accuracy.reset_states()

Epoch 1, Total Loss: 2.6845695972442627, Clf Loss: 2.684563636779785, Filter Loss: 0.5999999642372131, Seq Loss: 0.0, Accuracy Rate: 37.24%, Ones Portion: 1.0,             Test_Total_Loss: 2.3719279766082764, Test_Clf_Loss: 2.371922492980957, Test_Filter_Loss: 0.6000000238418579, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 37.17%, Test_Ones_Portion: 1.0
===MODEL WEIGHTS SAVED=== ./model/2019111101/model1 ./model/2019111101/model2
Epoch 2, Total Loss: 2.283953905105591, Clf Loss: 2.283947706222534, Filter Loss: 0.5999999642372131, Seq Loss: 0.0, Accuracy Rate: 39.59%, Ones Portion: 1.0,             Test_Total_Loss: 2.016993284225464, Test_Clf_Loss: 2.0169875621795654, Test_Filter_Loss: 0.6000000238418579, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 54.79%, Test_Ones_Portion: 1.0
===MODEL WEIGHTS SAVED=== ./model/2019111101/model1 ./model/2019111101/model2
Epoch 3, Total Loss: 1.936691403388977, Clf Loss: 1.93668532371521, Filter Loss: 0.5999999642372131, Seq Loss: 0.0, Accuracy Rate: 54.31%, On

Epoch 20, Total Loss: 1.2714006900787354, Clf Loss: 1.271395206451416, Filter Loss: 0.5999757647514343, Seq Loss: 0.00026189637719653547, Accuracy Rate: 69.00%, Ones Portion: 0.9998257160186768,             Test_Total_Loss: 1.6180757284164429, Test_Clf_Loss: 1.6180698871612549, Test_Filter_Loss: 0.599973738193512, TEST_Seq_Loss: 0.0003452335949987173, Test_Accuracy_Rate: 62.00%, Test_Ones_Portion: 0.9997702240943909
===MODEL WEIGHTS SAVED=== ./model/2019111101/model1 ./model/2019111101/model2
Epoch 21, Total Loss: 1.235358476638794, Clf Loss: 1.2353527545928955, Filter Loss: 0.5999699831008911, Seq Loss: 0.0003170114359818399, Accuracy Rate: 69.44%, Ones Portion: 0.9997889399528503,             Test_Total_Loss: 1.6256635189056396, Test_Clf_Loss: 1.6256574392318726, Test_Filter_Loss: 0.599964439868927, TEST_Seq_Loss: 0.0003989409015048295, Test_Accuracy_Rate: 62.10%, Test_Ones_Portion: 0.9997344017028809
===MODEL WEIGHTS SAVED=== ./model/2019111101/model1 ./model/2019111101/model2
Epoch

Epoch 39, Total Loss: 0.7406311631202698, Clf Loss: 0.7406253814697266, Filter Loss: 0.5996677279472351, Seq Loss: 0.0021481276489794254, Accuracy Rate: 80.62%, Ones Portion: 0.9985547661781311,             Test_Total_Loss: 1.9329887628555298, Test_Clf_Loss: 1.9329826831817627, Test_Filter_Loss: 0.5997004508972168, TEST_Seq_Loss: 0.0018036470282822847, Test_Accuracy_Rate: 64.23%, Test_Ones_Portion: 0.9987945556640625
Epoch 40, Total Loss: 0.7366213798522949, Clf Loss: 0.7366154789924622, Filter Loss: 0.5996575951576233, Seq Loss: 0.0021991077810525894, Accuracy Rate: 80.44%, Ones Portion: 0.9985189437866211,             Test_Total_Loss: 1.9884538650512695, Test_Clf_Loss: 1.9884480237960815, Test_Filter_Loss: 0.5996748208999634, TEST_Seq_Loss: 0.001921204966492951, Test_Accuracy_Rate: 62.63%, Test_Ones_Portion: 0.9987154603004456
Epoch 41, Total Loss: 0.7190544605255127, Clf Loss: 0.7190485596656799, Filter Loss: 0.5996213555335999, Seq Loss: 0.0024513318203389645, Accuracy Rate: 81.51%

Epoch 59, Total Loss: 0.44415462017059326, Clf Loss: 0.44414857029914856, Filter Loss: 0.599262535572052, Seq Loss: 0.004877943079918623, Accuracy Rate: 87.86%, Ones Portion: 0.9966984391212463,             Test_Total_Loss: 2.396456003189087, Test_Clf_Loss: 2.3964500427246094, Test_Filter_Loss: 0.5992951393127441, TEST_Seq_Loss: 0.004692421294748783, Test_Accuracy_Rate: 64.33%, Test_Ones_Portion: 0.9968185424804688
Epoch 60, Total Loss: 0.4435213506221771, Clf Loss: 0.4435153603553772, Filter Loss: 0.5992364883422852, Seq Loss: 0.005117140244692564, Accuracy Rate: 87.89%, Ones Portion: 0.9965356588363647,             Test_Total_Loss: 2.52487850189209, Test_Clf_Loss: 2.5248727798461914, Test_Filter_Loss: 0.5992677211761475, TEST_Seq_Loss: 0.004987022373825312, Test_Accuracy_Rate: 62.88%, Test_Ones_Portion: 0.9966148138046265
Epoch 61, Total Loss: 0.456148624420166, Clf Loss: 0.4561425745487213, Filter Loss: 0.5992261171340942, Seq Loss: 0.005161242093890905, Accuracy Rate: 87.32%, Ones 

Epoch 78, Total Loss: 0.3959627151489258, Clf Loss: 0.3959568440914154, Filter Loss: 0.5974922776222229, Seq Loss: 0.017309540882706642, Accuracy Rate: 89.23%, Ones Portion: 0.9879629015922546,             Test_Total_Loss: 2.345057487487793, Test_Clf_Loss: 2.3450512886047363, Test_Filter_Loss: 0.5973978638648987, TEST_Seq_Loss: 0.01943853311240673, Test_Accuracy_Rate: 65.97%, Test_Ones_Portion: 0.9864634275436401
Epoch 79, Total Loss: 0.38884663581848145, Clf Loss: 0.3888406455516815, Filter Loss: 0.5972655415534973, Seq Loss: 0.019170016050338745, Accuracy Rate: 89.27%, Ones Portion: 0.9865915179252625,             Test_Total_Loss: 2.4532053470611572, Test_Clf_Loss: 2.4531991481781006, Test_Filter_Loss: 0.5970709323883057, TEST_Seq_Loss: 0.021941686049103737, Test_Accuracy_Rate: 64.71%, Test_Ones_Portion: 0.9846252202987671
Epoch 80, Total Loss: 0.4165612757205963, Clf Loss: 0.4165552258491516, Filter Loss: 0.5969732403755188, Seq Loss: 0.021420957520604134, Accuracy Rate: 88.31%, One

...

Epoch 1918, Total Loss: 1.1716583967208862, Clf Loss: 1.1716583967208862, Filter Loss: 7.432824349962175e-05, Seq Loss: 0.9999966025352478, Accuracy Rate: 68.21%, Ones Portion: 9.603222861187533e-05,             Test_Total_Loss: 2.5509679317474365, Test_Clf_Loss: 2.5509679317474365, Test_Filter_Loss: 7.245733286254108e-05, TEST_Seq_Loss: 0.9999974370002747, Test_Accuracy_Rate: 57.55%, Test_Ones_Portion: 9.143014904111624e-05
Epoch 1919, Total Loss: 1.174544095993042, Clf Loss: 1.174544095993042, Filter Loss: 7.44793942430988e-05, Seq Loss: 0.9999964833259583, Accuracy Rate: 68.26%, Ones Portion: 9.606113599147648e-05,             Test_Total_Loss: 2.5672450065612793, Test_Clf_Loss: 2.5672450065612793, Test_Filter_Loss: 7.280157296918333e-05, TEST_Seq_Loss: 0.9999974370002747, Test_Accuracy_Rate: 56.78%, Test_Ones_Portion: 9.285464329877868e-05
Epoch 1920, Total Loss: 1.1715415716171265, Clf Loss: 1.1715415716171265, Filter Loss: 7.473077130271122e-05, Seq Loss: 0.9999966025352478, Accur

Epoch 1938, Total Loss: 1.167590618133545, Clf Loss: 1.167590618133545, Filter Loss: 7.490380812669173e-05, Seq Loss: 0.9999963641166687, Accuracy Rate: 68.12%, Ones Portion: 0.0001000878619379364,             Test_Total_Loss: 2.6259729862213135, Test_Clf_Loss: 2.6259729862213135, Test_Filter_Loss: 7.273626397363842e-05, TEST_Seq_Loss: 0.999997079372406, Test_Accuracy_Rate: 57.36%, Test_Ones_Portion: 9.633845183998346e-05
Epoch 1939, Total Loss: 1.1636748313903809, Clf Loss: 1.1636748313903809, Filter Loss: 7.450134580722079e-05, Seq Loss: 0.9999963641166687, Accuracy Rate: 68.40%, Ones Portion: 9.865090396488085e-05,             Test_Total_Loss: 2.6266510486602783, Test_Clf_Loss: 2.6266510486602783, Test_Filter_Loss: 7.231852214317769e-05, TEST_Seq_Loss: 0.9999974370002747, Test_Accuracy_Rate: 57.36%, Test_Ones_Portion: 9.314516501035541e-05
Epoch 1940, Total Loss: 1.158083438873291, Clf Loss: 1.158083438873291, Filter Loss: 7.466330862371251e-05, Seq Loss: 0.9999964833259583, Accurac

Epoch 1958, Total Loss: 1.2809113264083862, Clf Loss: 1.2809113264083862, Filter Loss: 7.490346615668386e-05, Seq Loss: 0.9999961256980896, Accuracy Rate: 65.87%, Ones Portion: 0.0001021577772917226,             Test_Total_Loss: 2.484998941421509, Test_Clf_Loss: 2.484998941421509, Test_Filter_Loss: 7.288978667929769e-05, TEST_Seq_Loss: 0.9999969601631165, Test_Accuracy_Rate: 57.50%, Test_Ones_Portion: 9.815233352128416e-05
Epoch 1959, Total Loss: 1.2667580842971802, Clf Loss: 1.2667580842971802, Filter Loss: 7.495789759559557e-05, Seq Loss: 0.9999961256980896, Accuracy Rate: 66.34%, Ones Portion: 0.00010239835683023557,             Test_Total_Loss: 2.4636590480804443, Test_Clf_Loss: 2.4636590480804443, Test_Filter_Loss: 7.276044198079035e-05, TEST_Seq_Loss: 0.999997079372406, Test_Accuracy_Rate: 57.12%, Test_Ones_Portion: 9.75064467638731e-05
Epoch 1960, Total Loss: 1.2396832704544067, Clf Loss: 1.2396832704544067, Filter Loss: 7.459682819899172e-05, Seq Loss: 0.9999962449073792, Accur

Epoch 1978, Total Loss: 1.1850982904434204, Clf Loss: 1.1850982904434204, Filter Loss: 7.56418303353712e-05, Seq Loss: 0.9999960064888, Accuracy Rate: 68.11%, Ones Portion: 0.00010291591752320528,             Test_Total_Loss: 2.5396265983581543, Test_Clf_Loss: 2.5396265983581543, Test_Filter_Loss: 7.263859151862562e-05, TEST_Seq_Loss: 0.999997079372406, Test_Accuracy_Rate: 57.21%, Test_Ones_Portion: 9.656514885136858e-05
Epoch 1979, Total Loss: 1.1829873323440552, Clf Loss: 1.1829873323440552, Filter Loss: 7.46228834032081e-05, Seq Loss: 0.9999961256980896, Accuracy Rate: 68.15%, Ones Portion: 0.0001013433065963909,             Test_Total_Loss: 2.529895067214966, Test_Clf_Loss: 2.529895067214966, Test_Filter_Loss: 7.274951349245384e-05, TEST_Seq_Loss: 0.9999969601631165, Test_Accuracy_Rate: 56.68%, Test_Ones_Portion: 9.723769471747801e-05
Epoch 1980, Total Loss: 1.1849775314331055, Clf Loss: 1.1849775314331055, Filter Loss: 7.513149466831237e-05, Seq Loss: 0.9999960064888, Accuracy Rat

Epoch 1998, Total Loss: 1.2007511854171753, Clf Loss: 1.2007511854171753, Filter Loss: 7.318711141124368e-05, Seq Loss: 0.9999968409538269, Accuracy Rate: 67.97%, Ones Portion: 8.815721230348572e-05,             Test_Total_Loss: 2.5341227054595947, Test_Clf_Loss: 2.5341227054595947, Test_Filter_Loss: 7.136039494071156e-05, TEST_Seq_Loss: 0.9999973177909851, Test_Accuracy_Rate: 57.31%, Test_Ones_Portion: 8.292699203593656e-05
Epoch 1999, Total Loss: 1.1867672204971313, Clf Loss: 1.1867672204971313, Filter Loss: 7.397493754979223e-05, Seq Loss: 0.9999963641166687, Accuracy Rate: 68.12%, Ones Portion: 9.461789159104228e-05,             Test_Total_Loss: 2.532308340072632, Test_Clf_Loss: 2.532308340072632, Test_Filter_Loss: 7.175374048529193e-05, TEST_Seq_Loss: 0.9999969601631165, Test_Accuracy_Rate: 57.50%, Test_Ones_Portion: 9.077605500351638e-05
Epoch 2000, Total Loss: 1.1938588619232178, Clf Loss: 1.1938588619232178, Filter Loss: 7.333277608267963e-05, Seq Loss: 0.9999964833259583, Accu

### Store process

In [1015]:
save_dir = './results/'+DateID+'/'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir, exist_ok=True)
saveR = save_dir + 'losses_metrics.xlsx'
data = {'train total loss':train_weighted_loss, 'train acc loss':train_loss_acc,
        'train filter loss':train_loss_filter,'train seq loss':train_loss_seq,
        'train acc rate':train_acc_rate, 'train ones num':train_ones_num,
        'test total loss':test_weighted_loss, 'test acc loss':test_loss_acc,
        'test filter loss':test_loss_filter, 'test seq loss': test_loss_seq,
        'test acc rate':test_acc_rate, 'test ones num':test_ones_num
       }
df = pd.DataFrame(data)
df.to_excel(saveR)
print(saveR)
df

./results/2019111101/losses_metrics.xlsx


,train total loss,train acc loss,train filter loss,train seq loss,train acc rate,train ones num,test total loss,test acc loss,test filter loss,test seq loss,test acc rate,test ones num
0,2.684570,2.684564,0.600000,0.000000,0.372397,1.000000,2.371928,2.371922,0.600000,0.000000,0.371733,1.000000
1,2.283954,2.283948,0.600000,0.000000,0.395884,1.000000,2.016993,2.016988,0.600000,0.000000,0.547919,1.000000
2,1.936691,1.936685,0.600000,0.000000,0.543099,1.000000,1.957407,1.957402,0.600000,0.000000,0.524685,1.000000
3,1.890775,1.890769,0.600000,0.000000,0.538378,1.000000,1.881698,1.881692,0.600000,0.000000,0.543562,1.000000
4,1.875519,1.875513,0.600000,0.000000,0.531598,1.000000,1.903455,1.903449,0.600000,0.000000,0.514037,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1.239868,1.239868,0.000073,0.999996,0.668644,0.000091,2.540167,2.540167,0.000071,0.999997,0.571152,0.000083
1996,1.210819,1.210819,0.000074,0.999996,0.676271,0.000091,2.531270,2.531270,0.000071,0.999997,0.571152,0.000083
1997,1.200751,1.200751,0.000073,0.999997,0.679661,0.000088,2.534123,2.534123,0.000071,0.999997,0.573088,0.000083
1998,1.186767,1.186767,0.000074,0.999996,0.681235,0.000095,2.532308,2.532308,0.000072,0.999997,0.575024,0.000091


## Inference
* Case study

In [10]:
'''
Get a list of keys from dictionary which has value that matches with any value in given list of values
'''
# def getKeysByValues(dictOfElements, listOfValues):
#     listOfKeys = list()
#     listOfItems = dictOfElements.items()
#     for item  in listOfItems:
#         if item[1] in listOfValues:
#             listOfKeys.append(item[0])
#     return  listOfKeys 

word_idx_pd = pd.DataFrame(data=word_index,index=["ID"]).T
word_idx_pd['ori_word'] = word_idx_pd.index 
word_idx_pd = word_idx_pd.set_index(["ID"])
print(word_idx_pd)

def getKeysByValues(listOfValues,dfOfElements=word_idx_pd):
    ori_word_li = []
    for item in listOfValues:
        try:
            key = item.numpy()
        except AttributeError:
            key = item
        if key == 0:
            continue
        ori_word = dfOfElements.loc[key][0]
        ori_word_li.append(ori_word)
    return ori_word_li

        ori_word
ID              
10996       mdbl
16260       fawc
12089    degussa
8803       woods
13796    hanging
...          ...
16258    rotting
10995       pods
2849       emery
30979  northerly
16259   onomichi

[30979 rows x 1 columns]


In [ ]:
def test_step(x,yc):
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #phase1: pred_imp ; phase2:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)
for test_text, test_labels in valid_ds:
    test_step(test_text, test_labels)

In [7]:
for test_text, test_labels in valid_ds:
    print(test_text)
    print(test_labels)
    break
#     test_step(test_text, test_labels)

NameError: name 'valid_ds' is not defined

In [11]:
convert_li = getKeysByValues(testX[0])
convert_li

['the',
 'in',
 'compared',
 'analysts',
 'etl',
 'at',
 'may',
 'said',
 'si',
 'offshore',
 'a',
 'but',
 'a',
 'ohio',
 'and',
 'publicly',
 'its',
 'sent',
 '380',
 'plan',
 'bank',
 'mln',
 '1',
 '756',
 'not',
 'note',
 'was',
 'finding',
 'eased',
 '80',
 'in',
 'compared',
 'analysts',
 'as',
 'main',
 'and',
 'origins',
 'be',
 'in',
 'borrowers',
 'said',
 'in',
 'hearings',
 'said',
 'becoming',
 '3',
 'as',
 'of',
 'was',
 'in',
 'worsen',
 'analysts',
 'that',
 'of',
 'offshore',
 'flight',
 'for',
 'during',
 'time',
 'official',
 'mln',
 'a',
 'for',
 'capital',
 'in',
 'par',
 'said',
 'in',
 'worsen',
 'sought',
 'bank',
 'sales',
 'commodity',
 'common',
 'above',
 'a',
 'merchants',
 'amounted',
 'already',
 'in',
 'capital',
 'mln',
 'in',
 'compared',
 'analysts',
 '3',
 'in',
 'worsen',
 '87',
 '41',
 'for',
 'hearings',
 'said',
 'becoming',
 'importers',
 'offshore',
 'flight',
 'be',
 'in',
 'has',
 'would',
 '24',
 '32',
 'firms',
 'of',
 'a',
 'only',
 'cattl

In [12]:
reuters.get_word_index()

{'mdbl': 10996,
 'fawc': 16260,
 'degussa': 12089,
 'woods': 8803,
 'hanging': 13796,
 'localized': 20672,
 'sation': 20673,
 'chanthaburi': 20675,
 'refunding': 10997,
 'hermann': 8804,
 'passsengers': 20676,
 'stipulate': 20677,
 'heublein': 8352,
 'screaming': 20713,
 'tcby': 16261,
 'four': 185,
 'grains': 1642,
 'broiler': 20680,
 'wooden': 12090,
 'wednesday': 1220,
 'highveld': 13797,
 'duffour': 7593,
 '0053': 20681,
 'elections': 3914,
 '270': 2563,
 '271': 3551,
 '272': 5113,
 '273': 3552,
 '274': 3400,
 'rudman': 7975,
 '276': 3401,
 '277': 3478,
 '278': 3632,
 '279': 4309,
 'dormancy': 9381,
 'errors': 7247,
 'deferred': 3086,
 'sptnd': 20683,
 'cooking': 8805,
 'stratabit': 20684,
 'designing': 16262,
 'metalurgicos': 20685,
 'databank': 13798,
 '300er': 20686,
 'shocks': 20687,
 'nawg': 7972,
 'tnta': 20688,
 'perforations': 20689,
 'affiliates': 2891,
 '27p': 20690,
 'ching': 16263,
 'china': 595,
 'wagyu': 16264,
 'affiliated': 3189,
 'chino': 16265,
 'chinh': 16266,
 '

,ori_word
ID,
10996,mdbl
16260,fawc
12089,degussa
8803,woods
13796,hanging
...,...
16258,rotting
10995,pods
2849,emery


In [1047]:
test_text[0][0].numpy()

1

In [1049]:
word_idx_pd.loc[test_text[0][0].numpy()][0]

'the'

In [1036]:
word_idx_pd

,ID
mdbl,10996
fawc,16260
degussa,12089
woods,8803
hanging,13796
...,...
rotting,16258
pods,10995
emery,2849
northerly,30979


In [714]:
np.array(train_weighted_loss[0])

array(None, dtype=object)

In [732]:
kk = np.array(test_accloss.result())
kk

array(2.3583207, dtype=float32)

In [735]:
test_accloss.result().numpy()

2.3583207

In [523]:
# model1.save('./model/emb_layer') #tf.keras.models.load_model
tf.saved_model.save(model1,'./model/emb_layer')
model3 = tf.keras.models.load_model('./model/emb_layer')

TypeError: unsupported callable

In [524]:
model3 = model1.load_weights('./model/emb_layer_weight')

AttributeError: 'CheckpointLoadStatus' object has no attribute 'summary'

In [527]:
model1.emb.set_weights(model3.emb.get_weights())

In [526]:
model3 = model1
model3.load_weights('./model/emb_layer_weight')
model3.summary()

Model: "base_model_1_58"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_58 (Masking)         multiple                  0         
_________________________________________________________________
glove_emb (Embedding)        multiple                  2138112   
_________________________________________________________________


ValueError: You tried to call `count_params` on common_extract, but the layer isn't built. You can build it manually via: `common_extract.build(batch_input_shape)`.

* 在某些參數的情況下0 1是train不起來的但是weight可以
    * alpha = 0.0, beta=0.6
    * init_w = tensorflow.keras.initializers.Constant(value=0.9), init_b = tensorflow.keras.initializers.Constant(value=0.7)
* 0/1 with emb比較容易overfit。如果是weight的比較沒那麼嚴重

* 不用sigmoid或是hard_sigmoid。改良relu + linear，並拆成兩個model，把round前面多加上clip
    * sigmoid中間的變化太快(一瞬間就會掉到0或是1)，改成relu在>0~無限大(linear為了還在0~1)再去clip再round，可以看到每個epoch的變化
* 1st phase的beta一定要>=0.6否則不會動，ones#都會是0
    * beta=1 (放0 1進去): weight init設成1也沒用，但把bias設成1就會一開始都是ones#=1了。weight=1 bias=0.2都匯市0，bias=0.3會是0.87。0.6/0.3都是0。0.8/0.3差不多是0.5(但是train很慢acc進步很慢就是了)。0.5/0.5是從0.01開始往上升 (0 1放進去會比較難train是因為它的變化量太大，一下就是有或沒有，所以clf可能學不好，但如果是weight每次gradient進步的都是一小點就會比較容易上升)
    * beta=0.6 (放0 1進去): 0.8/0.5都是0。0.9/0.8 從0.9一直到0。0.9/0.6差不多是從0.5但又有時候到0.7都是0.0(很難train，Nadam換個opt有時候沒用。EX變成adam 0.9/0.8才有0.96開始但如果0.9/0.75變成0開始。Rmsprop 0.9/0.8又是從0.95開始往下)。但如果都改成傳入weight就都沒問題。ones一開始大概0.5 weight平均，也不會卡住
* 建議: 先訓練embedding weight matrix，但是要看goal是要怎樣的matrix
* 其實他不管幾%都會train得很好，除非固定embedding，或是用更弱的clf
* 若設定alpha，就像是regularizer term (penalty)，設越大drop越多
* 一開始先很多ones，再越來越少個

* 同一個opt若加入transform就會train不起來
* 兩個不同的opt加入transform也會train不起來 (persistent、non-persis都不行)，且與BN無關